In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

In [4]:
df = pd.read_csv('../output/gesture_data.csv')

In [5]:
df.head()

,0,1,2,3,4,5,6,7,gesture
0,5.44,7.20,2.48,5.98,2.90,4.10,2.74,2.80,大拇指
1,2.46,8.28,2.74,1.96,1.64,1.60,2.38,1.64,大拇指
2,2.50,11.18,2.96,2.28,1.70,1.72,2.14,1.90,大拇指
3,2.36,11.26,3.68,2.72,1.86,1.70,1.98,1.50,大拇指
4,2.94,12.76,2.92,2.22,1.90,1.94,1.84,2.08,大拇指


## 数据观察

In [6]:
df1 = df.set_index('gesture').stack().reset_index()
df1.columns= ['g','n','v']

In [22]:
def data_process1(df):
    """

    :param df:
    :return:
    """
    le = preprocessing.LabelEncoder()
    labels = le.fit_transform(df['gesture'])
    cols = [str(i) for i in range(8)]
    data = df[cols].values
    return data, labels

In [23]:
data, labels = data_process1(df)

In [9]:
# plt.figure(figsize = (15,10))
# sns.barplot(x='g',y='v',data = df1, hue = 'n')

In [30]:
clf = RandomForestClassifier()
from sklearn.model_selection import cross_val_score
cross_val_score(clf, data, labels, cv=5)

array([0.89880952, 0.95238095, 0.94047619, 0.9702381 , 0.94047619])